## Median Subtraction

In [27]:
import numpy as np

In [28]:
a = np.array([[11, 12, 13, 2530, 11], [16, 18, 4800, 17, 18]])

In [30]:
m = np.median(a, axis=1, keepdims=True)

m

In [31]:
np.shape(m)

(2, 1)

In [32]:
np.shape(a)

(2, 5)

In [33]:
b = a - m

In [34]:
b

array([[-1.000e+00,  0.000e+00,  1.000e+00,  2.518e+03, -1.000e+00],
       [-2.000e+00,  0.000e+00,  4.782e+03, -1.000e+00,  0.000e+00]])

## Log Stretch

In [9]:
from math import log10, floor

def log_stretch_transform(black_point, saturation_range):
    
    log_saturation_range = log10(saturation_range)
    
    def fn(pixel_value):
        pixel_value -= black_point
        if pixel_value <= 1.0:
            return 0
        else:
            log_pixel_value = log10(pixel_value)
            if log_pixel_value >= log_saturation_range:
                return 255;
            else:
                return floor(256 * log_pixel_value / log_saturation_range)
        
    return fn


In [45]:
transform = np.vectorize(log_stretch_transform(0, 5000))

In [46]:
b_transformed = transform(b)

In [47]:
b_transformed

array([[  0,   0,   0, 235,   0],
       [  0,   0, 254,   0,   0]])

## SN 2022abik 2022-12-1415

### Observing Notes

Another round 30 60-second exposures each in g', r', and i', this time, proceeding across series rather than doing all of g' first.

TheSkyX crashed when approximately 80 of the planned 90 images had been taken.

### Calibration Photos

The calibration photos are in 2022-11-3001.

The bias photos should be discarded in favor of the 1-second and 30-second darks. It turns out TheSkyX ignores the exposure time
set on bias photos and does some very rapid exposure. I would not have noticed this if I had not needed some 30-second
bias files.

The 30-second darks are needed because the i' flats required 30 seconds to get a reasonable value.

In [2]:
import os
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection, Combiner, combine, subtract_dark, flat_correct
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# for an analyses of an observation run done with only a single filter,
# only the observation_date and the exposure times
# will need to be changed

observation_date = '2022-12-1415'
calibration_date = '2022-11-3001'
light_exposure = 30 * u.second
dark_exposure = light_exposure  # our method presumes this equality
flat_exposure = 1 * u.second
bias_exposure = flat_exposure  # our method presumes this equality

In [9]:
# directories where the observation and calibration photos being analyzed is kept

observation_directory = os.path.join(os.path.expanduser('~'), 'Transients', observation_date)
calibration_directory = os.path.join(os.path.expanduser('~'), 'Transients', calibration_date)

In [14]:
# subdirectories -- relative to observation and calibration directories

g_light_directory = os.path.join(observation_directory, 'g')
r_light_directory = os.path.join(observation_directory, 'r')

# the 60-second darks
dark_directory = os.path.join(calibration_directory, 'dark')

# the 1-second g and r flats
g_flat_directory = os.path.join(calibration_directory, 'flat', 'g')
r_flat_directory = os.path.join(calibration_directory, 'flat', 'r')

# the 1-second darks/biases
bias_directory = os.path.join(calibration_directory, 'bias')

# the aligned directories arelights = [CCDData.read(file, unit=u.adu) for file in light_files]
darks = [CCDData.read(file, unit=u.adu) for file in dark_files]
flats = [CCDData.read(file, unit=u.adu) for file in flat_files]
biases = [CCDData.read(file, unit=u.adu) for file in bias_files] *written to* not read from

g_aligned_directory = os.path.join(g_light_directory, 'aligned')
r_aligned_directory = os.path.join(r_light_directory, 'aligned')

if not os.path.exists(g_aligned_directory):
    os.makedirs(g_aligned_directory)
    
if not os.path.exists(r_aligned_directory):
    os.makedirs(r_aligned_directory)

In [17]:
g_lights = [CCDData.read(file, unit=u.adu) for file in g_light_files]
r_lights = [CCDData.read(file, unit=u.adu) for file in r_light_files]
darks = [CCDData.read(file, unit=u.adu) for file in dark_files]
g_flats = [CCDData.read(file, unit=u.adu) for file in g_flat_files]
r_flats = [CCDData.read(file, unit=u.adu) for file in r_flat_files]
biases = [CCDData.read(file, unit=u.adu) for file in bias_files]

Set OBSGEO-Y to -4483197.997 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851226.198 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483197.997 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851226.198 from OBSGEO-[LBH]'.
